In [198]:
import pandas as pd
import os


DATA_ROOT = os.path.join(os.getcwd(), 'data')

df_colors = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painiting - Colors Used'))
df_colors['TITLE'] = df_colors.apply(
    lambda r: r['painting_title'].upper(), axis=1)
df_colors['EPISODE'] = df_colors.apply(
    lambda r: f"S{r['season']:02}E{r['episode']:02}", axis=1)

df_subjects = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painiting - Subject Matter'))
# update column - remove surrounding quotes from TITLE
df_subjects['TITLE'] = df_subjects.apply(lambda r: r['TITLE'][1:-1], axis=1)

df_dates = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painting - Episode Dates'), delimiter="\t", header=None, names=['src'])

# parse df_dates (not actually CSV-- parse each row of file as col in df)
df_dates['date'] = df_dates.apply(
    lambda r: r['src'][r['src'].find('(')+1:r['src'].find(')')].strip(), axis=1)
df_dates['title'] = df_dates.apply(
    lambda r: r['src'].split(r['date'])[0][:-1].strip(), axis=1
)
df_dates['note'] = df_dates.apply(
    lambda r: r['src'].split(r['date'])[1][1:-1].strip(), axis=1
)
df_dates['TITLE'] = df_dates.apply(lambda r: r['title'].upper(), axis=1)

# get complete TITLE -> EPISODE xwalk from union of matches in df_colors/df_subjects


df_titles = pd.concat([df_colors[['TITLE', 'EPISODE']],
                      df_subjects[['TITLE', 'EPISODE']]])
# drop missing matches
df_titles.dropna(inplace=True)
# drop duplicates
df_titles.drop_duplicates(inplace=True)


In [191]:
df_titles.loc[df_titles['TITLE'].str.contains('SHADES OF GREY')]

,TITLE,EPISODE
16,SHADES OF GREY,S02E04


In [221]:
# show missing titles
def getDatesForTitles(r):
    xwalk = {
        'EVERGREENS AT SUNSET': 'EVERGREEN AT SUNSET',
        'MOUNTAIN PATH': 'MOUNTAIN PASS',
        'SHADES OF GREY': 'SHADES OF GRAY',
        'EVENING AT SUNSET': 'EVENING SUNSET',
        'HALF-OVAL VIGNETTE': 'HALF OVAL VIGNETTE',
        'OLD OAK TREE': 'THE OLD OAK TREE',
        'CABIN AT TRIALS END': 'CABIN AT TRIAL\'S END',
        'GRAY MOUNTAIN': 'GREY MOUNTAIN',
        'THE OLD HOME PLACE': 'OLD PLACE HOME',
        'WOODMAN\'S RETREAT': 'WOODSMAN\'S RETREAT',
    }
    title = r['TITLE']
    if title in xwalk:
        title = xwalk[title]
    matches = df_dates.loc[df_dates['TITLE'].str.contains(title)]
    if matches.empty:
        return None
    return matches.iloc[0]['date']


def getEpisodeForDate(r):
    date = r['date']
    title = r['TITLE']
    if date:
        matches = df_titles.loc[df_titles['date'] == date]
        if not matches.empty:
            return matches.iloc[0]['EPISODE']
    if title:
        matches = df_titles.loc[df_titles['TITLE'].str.contains(title)]
        if not matches.empty:
            return matches.iloc[0]['EPISODE']
    return None


df_titles['date'] = df_titles.apply(getDatesForTitles, axis=1)
df_dates['EPISODE'] = df_dates.apply(getEpisodeForDate, axis=1)


In [222]:


df_dates.loc[df_dates['EPISODE'].isnull()]


,src,date,title,note,TITLE,EPISODE
264,"Cabin at Trial's End (October 3, 1990)","October 3, 1990",Cabin at Trial's End,,CABIN AT TRIAL'S END,None


In [16]:
print("The Joy Of Painiting - Episode Dates:")
df_episode_dates.head()


The Joy Of Painiting - Episode Dates:


,0
0,"A Walk in the Woods (January 11, 1983)"
1,"Mount McKinley (January 11, 1983)"
2,"Ebony Sunset (January 18, 1983)"
3,"Winter Mist (January 25, 1983)"
4,"Quiet Stream (February 1, 1983)"
